In [1]:
import qgrid
import pandas as pd
import numpy as np
randn = np.random.randn
df3 = pd.DataFrame({
    'A' : 1.,
    'B' : pd.Timestamp('20130102'),
    'C' : pd.Series(randn(8),index=list(range(8)),dtype='float32'),
    'D' : np.array([3] * 8,dtype='int32'),
    'E' : pd.Categorical(["washington", "adams", "madison", "lincoln","jefferson", "hamilton", "roosevelt", "kennedy"]),
    'F' : 'foo' })
df3

,A,B,C,D,E,F
0,1.0,2013-01-02,-1.008362,3,washington,foo
1,1.0,2013-01-02,-0.293146,3,adams,foo
2,1.0,2013-01-02,2.413145,3,madison,foo
3,1.0,2013-01-02,1.820680,3,lincoln,foo
4,1.0,2013-01-02,0.874463,3,jefferson,foo
5,1.0,2013-01-02,-0.162878,3,hamilton,foo
6,1.0,2013-01-02,0.962699,3,roosevelt,foo
7,1.0,2013-01-02,-1.575095,3,kennedy,foo


In [2]:
qgrid.show_grid(df3)

A Jupyter Widget